In [6]:
import requests
import json

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, AutoModelForCausalLM


In [51]:
#model used for captioning image input query
input_processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
input_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco").to("cuda")
#model used for object detection
dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny").to("cuda")

Use below for regular use

In [55]:
input_url = "input-data/cat.jpg"
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [56]:
#opens the image input, and captions it
image = Image.open(input_url)

pixel_values = input_processor(images=image, return_tensors="pt").pixel_values.to("cuda")

generated_ids = input_model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = input_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)


image = Image.open(requests.get(image_url, stream=True).raw)
# Uses generated caption to detect
text_labels = [[generated_caption, "null"]]

inputs = dino_processor(images=image, text=text_labels, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = dino_model(**inputs)

results = dino_processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

full_results = []

# Retrieve the first image result
result = results[0]
for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {labels} with confidence {round(score.item(), 3)} at location {box}")
    full_results.append(box)

red cat sitting on a white background
Detected red cat with confidence 0.554 at location [345.69, 23.8, 637.67, 372.95]


Use below for COCO evaluation.

In [57]:
coco_dataset = open("data/instances_val2017.json")
coco_json = json.load(coco_dataset)
coco_images = coco_json["images"]
coco_annotations = coco_json["annotations"]
coco_categories = coco_json["categories"]
input_url = []
image_url = []
for anno in coco_annotations:
    for images in coco_images:
        if anno["image_id"] == images["id"]:
            image_url.append(images["coco_url"])
            break
    for categories in coco_categories:
        if anno["category_id"] == categories["id"]:
            input_url.append("input-data/" + categories["name"] + ".jpg")
            break

In [58]:
print(len(input_url))
print(len(image_url))

36781
36781


In [ ]:
full_results = []
for x in range(len(input_url)):
    #opens the image input, and captions it
    image = Image.open(input_url[x])

    pixel_values = input_processor(images=image, return_tensors="pt").pixel_values.to("cuda")

    generated_ids = input_model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = input_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


    image = Image.open(requests.get(image_url[x], stream=True).raw)
    # Uses generated caption to detect
    text_labels = [[generated_caption, "null"]]

    inputs = dino_processor(images=image, text=text_labels, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = dino_model(**inputs)

    results = dino_processor.post_process_grounded_object_detection(
        outputs,
        inputs.input_ids,
        box_threshold=0.4,
        text_threshold=0.3,
        target_sizes=[image.size[::-1]]
    )


    # Retrieve the first image result
    result = results[0]
    for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
        box = [round(x, 2) for x in box.tolist()]
    formatted_results = {
        "image_id": coco_annotations[x]["image_id"],
        "category_id": coco_annotations[x]["category_id"],
        "bbox": box,
        "score": round(score.item(), 3)
    }
    if x%10 == 0:
        print(str(x) + " out of " + str(len(image_url)))
    full_results.append(formatted_results)
full_results = json.dumps(full_results)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
with open("results.json", "w") as f:
  f.write(full_results)